In [1]:
import networkx as nx
from geopandas import sjoin
import re
import pandas as pd
from geopandas import GeoDataFrame
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.cm as cm,numpy as np
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import numpy as np
from random import sample
import scipy.sparse as sparse
from __future__ import division

In [2]:
with open("d_mob_study_out.p", "rb") as f:
    d_mob_study_out = pickle.load(f)
geometry=[]
geo_real_home=[]
for usr,pos in tqdm(d_mob_study_out['new_dic_real_home'].items()):
    for single_pos in pos:
        geo_pt=Point(single_pos[1],single_pos[0])
        geo_real_home.append(usr)
        geometry.append(geo_pt)
df=pd.DataFrame(list(zip(geo_real_home,geometry)),columns=['usr','geometry'])
df=df.dropna(how='any')
gdf_real_home = GeoDataFrame(df,geometry=geometry)

100%|██████████| 29677/29677 [00:26<00:00, 1109.98it/s]


In [3]:
import warnings
warnings.filterwarnings("ignore")
df_insee=pd.read_csv('/datastore/complexnet/jlevyabi/geoloc/final.csv')
pat = re.compile(r'''(-*\d+\.\d+ -*\d+\.\d+);*''');new_geo=[]
for geo in tqdm(df_insee.geometry_y):
    matches = pat.findall(geo)
    if matches:
        lst = Polygon([tuple(map(float, m.split())) for m in matches])
    else:
        lst=None
    new_geo.append(lst)

presentable_insee_df=df_insee.drop(["geometry","geometry_y","Unnamed: 0","idINSPIRE",
                                    "geometry_x","nbcar_x","id_x","id_y","idk"],axis=1)
presentable_insee_df=presentable_insee_df.drop("center",axis=1)
presentable_insee_df=presentable_insee_df.convert_objects(convert_numeric=True)
inter= presentable_insee_df.astype(float).round(3)
presentable_insee_df =inter.dropna(how='any')
val_idx = inter.index.isin(presentable_insee_df.index)
presentable_insee_df['geometry']=[new_geo[it] for it,x in enumerate(val_idx) if x]
geo_insee = GeoDataFrame(presentable_insee_df)
geom=geo_insee.geometry
geo_insee["center"]=[poly.centroid for poly in tqdm(geo_insee.geometry)]
 
gdf_real_home.crs = geo_insee.crs
usrs_with_insee_income = sjoin(gdf_real_home,geo_insee, how="inner", op='within')

dic_insee={}
for it,row in tqdm(usrs_with_insee_income.iterrows()):
    if row.usr not in dic_insee:
        dic_insee[row.usr]=[]
    income=row.income
    own_rat=(row.men_prop+0.0)/row.ind_r
    dic_insee[row.usr].append([row.geometry,income,own_rat])
for key,val in tqdm(dic_insee.items()):
    dic_insee[key]=GeoDataFrame(val,columns=["geotw","income","owner_ratio"])

more_locs=[v.shape[0] for v in (dic_insee.values()) ]
dic_insee_avg=pd.DataFrame([v.mean() for v in (dic_insee.values()) ],columns=["income","owner_ratio"])
dic_insee_avg["usr"]=list(dic_insee.keys())
get_mode = lambda x: x.value_counts(dropna=True).index[0]
get_mode.__name__ = "most frequent"
dic_insee_mode=pd.DataFrame([v[["income","owner_ratio"]].agg(get_mode) for v in (dic_insee.values())],
                            columns=["income","owner_ratio"])
dic_insee_mode["usr"]=list(dic_insee.keys())

100%|██████████| 2277947/2277947 [01:42<00:00, 22298.28it/s]
451030it [04:19, 1736.32it/s]
100%|██████████| 28035/28035 [01:13<00:00, 382.95it/s]


In [4]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('/datastore/complexnet/jlevyabi/network_representation/python_scripts/')
import helpers_ses_prediction as hsp

dic_insee_avg["income_class"]=hsp.ses_classify(dic_insee_avg.income)
dic_insee_avg["owner_ratio_class"]=hsp.ses_classify(dic_insee_avg.owner_ratio)
dic_insee_avg['ses_class'] = list(zip(dic_insee_avg.income_class, dic_insee_avg.owner_ratio_class))
dic_insee_avg.usr=[str(usr) for usr in dic_insee_avg.usr]

In [6]:
dic_insee_avg.tail(1) 

,income,owner_ratio,usr,income_class,owner_ratio_class,ses_class
28034,19711.682,0.363636,92534651,1,2,"(1, 2)"


In [12]:
und_file="/datastore/complexnet/jlevyabi/network_representation/data_files/network_files/mutual_mention_network.txt"
fh=open(und_file, 'rb')
G=nx.read_weighted_edgelist(fh,delimiter=',',nodetype=str)
fh.close()
print("Network Loaded ...")
geo_usrs=[int(x) for x in list(G.nodes()) if len(x)!=0]

Network Loaded ...


In [14]:
nodes_with_labels=list(set([(x) for x in dic_insee_avg.usr
                           ]).intersection(set(G.nodes())))
labels=np.array([int(dic_insee_avg[dic_insee_avg.usr==node].income_class) for node in nodes_with_labels])
node2label={node:label for node,label in zip(nodes_with_labels,labels) }
order_nodes={node:it for it,node in enumerate(G.nodes()) if len(node)>0}

In [68]:
int2label={order_nodes[node]:int(label) 
           for node,label in tqdm(node2label.items())}
G_clear=nx.Graph()
for u,v in tqdm(G.edges_iter()):
    if len(u)==0 or len(v)==0:
        continue
    G_clear.add_edge(order_nodes[u],order_nodes[v],
                     weight=G[u][v]["weight"])

100%|██████████| 27130/27130 [00:01<00:00, 14436.27it/s]
11929231it [03:33, 55890.68it/s]


In [70]:
## set up training and test set
n=len(G_clear.nodes()) # number of node in the network
m=len(G_clear.edges()) # number of edges in the network
c=3 # number of discrete class labels

l=int(0.8*len(nodes_with_labels)) # number of nodes in training set
u=n-l # number of nodes in the test set
L=sample([order_nodes[x] for x in nodes_with_labels],l) 
U=np.setdiff1d(G_clear.nodes(),L)

# n x c matrix indicating known class label information
# Y_ik = 1 if we know that node i belongs to class k and 
#        0 if we know it does not.  Otherwise we set to 1/c (indicates uncertainty)
Y=np.ones((n,c))/c
Y[L,:]=0
Y[L,[int2label[node] for node in L]]=1

In [75]:
# construct normalised laplacian L
A=nx.adjacency_matrix(G_clear)
#A=A+A.T
degree=np.array(A.sum(1)).flatten()
D=sparse.diags(np.power(degree+1e-200,-.5),0,format='csc')
Lnorm=D.dot(A).dot(D)

In [77]:
# perform 2-step label propagation
steps = 2
alpha = 0.1 # damping factor (typically this is a good setting)
F=Y.copy() # initialise predictions F
diff=1 
it=0
F_old=-1
while (diff>1e-6 and it<200): 
    for si in range(steps):
        F=Lnorm.T.dot(F)
    F =  (1-alpha)*Y + alpha*F

    diff = np.sum(np.abs(F-F_old))
    it+=1
    F_old=F.copy()

In [108]:
# test accuracy
predicted_labels = np.argmax(F,1)
comp=np.array([(x==y) for x,y in zip(predicted_labels,labels)])
accuracy = np.sum(comp)
print ('accuracy', accuracy)

accuracy 9082


In [109]:
len(L)

21704